In [2]:
%env CUDA_DEVICE_ORDER=PCI_BUS_ID
%env CUDA_VISIBLE_DEVICES=0

import tensorflow as tf
import tensorflow_addons as tfa
import tensorflow.keras as keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, Activation, Flatten, Dropout, BatchNormalization
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.datasets import cifar10
from tensorflow.keras import regularizers, optimizers
import numpy as np

physical_devices = tf.config.experimental.list_physical_devices('GPU')
assert len(physical_devices) > 0, "Not enough GPU hardware devices available"
config = tf.config.experimental.set_memory_growth(physical_devices[0], True)


### full code-add instance norm ###

(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

#z-score
mean = np.mean(x_train,axis=(0,1,2,3))
std  = np.std(x_train,axis=(0,1,2,3))
x_train = (x_train-mean)/(std+1e-7)
x_test  = (x_test-mean)/(std+1e-7)

baseMapNum = 32
weight_decay = 1e-5

model = Sequential()
model.add(Conv2D(32, (3,3), padding='same', activation='relu', kernel_regularizer=regularizers.l1_l2(weight_decay, weight_decay), input_shape=x_train.shape[1:]))
model.add(tfa.layers.InstanceNormalization())
model.add(BatchNormalization())
model.add(Conv2D(32, (3,3), padding='same', activation='relu', 
                 kernel_regularizer=regularizers.l1_l2(weight_decay, weight_decay)))
model.add(tfa.layers.InstanceNormalization())
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.3))

model.add(Conv2D(64, (3,3), padding='same', activation='relu', kernel_regularizer=regularizers.l1_l2(weight_decay, weight_decay)))
model.add(tfa.layers.InstanceNormalization())
model.add(BatchNormalization())
model.add(Conv2D(64, (3,3), padding='same', activation='relu', kernel_regularizer=regularizers.l1_l2(weight_decay, weight_decay)))
model.add(tfa.layers.InstanceNormalization())
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.3))

model.add(Conv2D(128, (3,3), padding='same', activation='relu', kernel_regularizer=regularizers.l1_l2(weight_decay, weight_decay)))
model.add(tfa.layers.InstanceNormalization())
model.add(BatchNormalization())
model.add(Conv2D(128, (3,3), padding='same', activation='relu', kernel_regularizer=regularizers.l1_l2(weight_decay, weight_decay)))
model.add(tfa.layers.InstanceNormalization())
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Conv2D(128, (3,3), padding='same', activation='relu', kernel_regularizer=regularizers.l1_l2(weight_decay, weight_decay)))
model.add(tfa.layers.InstanceNormalization())
model.add(BatchNormalization())
model.add(Conv2D(128, (3,3), padding='same', activation='relu', kernel_regularizer=regularizers.l1_l2(weight_decay, weight_decay)))
model.add(tfa.layers.InstanceNormalization())
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Flatten())
model.add(Dense(128))
model.add(Dense(10, activation='softmax'))

#model.summary()

#training
batch_size = 256
epochs = 150
steps = x_train.shape[0] // batch_size

boundaries = [steps*70, steps*110]
values = [0.001, 0.0005, 0.0001]
schedules = keras.optimizers.schedules.PiecewiseConstantDecay(boundaries, values)
opt_adam  = keras.optimizers.Adam(learning_rate=schedules)


# data augmentation
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
test_dataset  = tf.data.Dataset.from_tensor_slices((x_test, y_test))

def process_data(image, label):
    if tf.random.uniform(()) > 0.5:
        image = tf.image.resize(image, (38, 38))
        image = tf.image.random_crop(image, size=[32, 32, 3])
        
    if tf.random.uniform(()) > 0.5:
        image = tf.image.flip_left_right(image)

    return image, label

train_batches = (train_dataset.shuffle(256*4)
                              .map(process_data, num_parallel_calls=tf.data.experimental.AUTOTUNE)
                              .batch(batch_size)
                              .prefetch(tf.data.experimental.AUTOTUNE) )

test_dataset = (test_dataset.batch(batch_size))

model.compile(loss='sparse_categorical_crossentropy', optimizer=opt_adam, metrics=['accuracy'])
history = model.fit(train_batches, validation_data=test_dataset, epochs=epochs, verbose=1)

env: CUDA_DEVICE_ORDER=PCI_BUS_ID
env: CUDA_VISIBLE_DEVICES=0
Epoch 1/150
196/196 [==============================] - 13s 68ms/step - loss: 2.3468 - accuracy: 0.3242 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 2/150
196/196 [==============================] - 7s 37ms/step - loss: 1.6651 - accuracy: 0.4580 - val_loss: 1.7566 - val_accuracy: 0.4690
Epoch 3/150
196/196 [==============================] - 7s 37ms/step - loss: 1.4723 - accuracy: 0.5348 - val_loss: 1.3421 - val_accuracy: 0.5817
Epoch 4/150
196/196 [==============================] - 7s 37ms/step - loss: 1.3401 - accuracy: 0.5814 - val_loss: 1.1876 - val_accuracy: 0.6441
Epoch 5/150
196/196 [==============================] - 7s 37ms/step - loss: 1.2238 - accuracy: 0.6265 - val_loss: 1.0678 - val_accuracy: 0.6835
Epoch 6/150
196/196 [==============================] - 7s 37ms/step - loss: 1.1352 - accuracy: 0.6599 - val_loss: 1.0398 - val_accuracy: 0.7064
Epoch 7/150
196/196 [==============================] - 7s 37ms/st

196/196 [==============================] - 7s 38ms/step - loss: 0.6330 - accuracy: 0.8719 - val_loss: 0.6387 - val_accuracy: 0.8797
Epoch 58/150
196/196 [==============================] - 7s 38ms/step - loss: 0.6288 - accuracy: 0.8727 - val_loss: 0.6174 - val_accuracy: 0.8824
Epoch 59/150
196/196 [==============================] - 7s 38ms/step - loss: 0.6305 - accuracy: 0.8736 - val_loss: 0.6351 - val_accuracy: 0.8777
Epoch 60/150
196/196 [==============================] - 7s 38ms/step - loss: 0.6226 - accuracy: 0.8748 - val_loss: 0.6388 - val_accuracy: 0.8773
Epoch 61/150
196/196 [==============================] - 7s 38ms/step - loss: 0.6270 - accuracy: 0.8748 - val_loss: 0.6225 - val_accuracy: 0.8822
Epoch 62/150
196/196 [==============================] - 7s 38ms/step - loss: 0.6241 - accuracy: 0.8756 - val_loss: 0.6339 - val_accuracy: 0.8771
Epoch 63/150
196/196 [==============================] - 7s 38ms/step - loss: 0.6246 - accuracy: 0.8768 - val_loss: 0.6229 - val_accuracy: 0.882

In [3]:
import numpy as np
accs = np.array(history.history['accuracy'])*100
for i in range(150):
    print(accs[i])

32.418
45.798
53.482002
58.143997
62.646
65.99
68.342
70.371994
71.992
73.186
74.757996
75.743996
76.542
77.236
77.948
78.715996
79.144
79.848
80.022
80.593994
81.048
81.383995
81.632
82.037994
82.292
82.792
83.285995
83.112
83.516
83.678
83.954
84.038
84.348
84.454
84.715996
84.598
84.774
85.299995
85.136
85.548004
85.332
85.836
85.7
86.02
86.21
86.228004
86.236
86.189995
86.31
86.592
86.596
86.872
86.992004
86.822
86.644
87.082
87.187996
87.268
87.362
87.476
87.478
87.558
87.67799
87.742004
87.88
87.91
87.696
88.06
88.147995
88.3
89.526
89.796
89.75
89.814
89.911995
90.112
90.200005
90.314
90.15
90.236
90.064
90.384
90.46
90.472
90.402
90.58
90.866
90.502
90.675995
90.688
90.682
90.925995
90.862
90.854004
90.696
90.792
90.764
91.006
90.804
91.024
91.022
90.958
91.072
90.922
91.21
90.978004
91.04401
91.122
91.198
91.496
92.25
92.45
92.368004
92.78
92.628006
92.72
92.822
92.826004
92.906
92.96
93.036
92.738
93.014
93.034
93.096
93.252
93.09
93.22
93.018
93.212006
93.264
93.146
93.264
9

In [4]:
val_as = np.array(history.history['val_accuracy'])*100
for i in range(150):
    print(val_as[i])

40.43
46.9
58.170002
64.41
68.35
70.64
73.909996
72.259995
75.81
76.200005
77.020004
78.78999
79.29
80.76
80.65
80.909996
80.61
81.53
82.5
82.91
83.41
83.47
83.18
84.3
85.02
84.11
84.88
84.58
85.59
84.85
86.14
85.47
85.579994
86.02
86.65
86.229996
86.43
87.01
86.909996
86.94
86.82
86.479996
85.829994
87.09
86.81
86.93
87.32
87.5
86.83
87.52
87.56
87.49
87.57
87.82
87.5
87.75
87.97
88.24
87.77
87.73
88.22
87.71
88.26
88.9
86.95
87.98
88.5
88.15
88.340004
89.840004
89.25
89.560005
89.83
89.76
90.05
89.62
90.060005
89.87
89.83
89.93
90.35
89.87
89.93
90.29
90.26
90.32
90.090004
89.69
90.21
90.36
90.04
90.48
90.12
90.4
89.76
90.0
90.020004
90.48
90.08
90.3
90.11
90.08
90.020004
90.39
90.43
90.57
90.33
90.08
90.03
90.81
90.97
91.03
90.89
91.06
91.06
91.02
90.99
91.189995
91.14
91.24
91.07
91.28
91.25
91.439995
91.439995
91.28
91.259995
91.39
91.329994
91.38
91.34
91.399994
91.38
91.329994
91.5
91.409996
91.36
91.439995
91.46999
91.46999
91.46
91.38
91.46999
91.46
91.57
91.5
91.27
91.36
91.3